In [1]:
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib
import pandas as pd
import pylab as pl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import statsmodels.formula.api as smf
%pylab inline

# This client code can run on Python 2.x or 3.x.  Your imports can be
# simpler if you only need one of those.
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode
    
from pandas.io.json import json_normalize

Populating the interactive namespace from numpy and matplotlib


In [8]:
# Yelp Fusion no longer uses OAuth as of December 7, 2017.
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app
API_KEY= "3G5x4RyMW1DWexU5hqQBgYa-AYv57HjXJ8MtnoB62rNW_SiKd1kyvokMtwYncCMtquPen7YjcdUIL_5aaaiazKvgclMFeoNEnYSMwcf8owl0wboMIO13JOSMsLPYWnYx" 


# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 50

def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

#     print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

def search(api_key, term, location, categories, offset,price):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT,
        'offset': offset,
        'categories': categories,
        'price':price
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

In [23]:
def getrestaurant(cuisine,loc='New York',cat=None,price=[1,2,3,4]):
    if cat == None:
        cat = cuisine
    cui = pd.DataFrame()
    for i in range(20):
        try:
            res = search(API_KEY,cuisine,loc,cat,50*i,price)['businesses']
    #         if res == []:
    #             break
            k = json_normalize(res)
            cui = pd.concat([cui,k],ignore_index=True)
        except:
            continue
#         if len(k) < 50:
#             break
    
#     cat = set(['Japanese','Sushi Bars','Japanese Curry','Izakaya'])
#     fact = pd.DataFrame()
#     for x in cui.index:
#         for e in cui.loc[x,'categories']:
#             if e['title'] == cat:
#                 fact = pd.concat([fact,cui[cui.index==x]])
#                 break
    cui.to_csv(cuisine+'_'+loc+'_'+str(price)+'.csv',encoding='utf-8')
    print(cuisine,loc,price,len(cui))

In [43]:
def getrestaurant(cuisine,loc='New York',cat=None,price=[1,2,3,4]):
    if cat == None:
        cat = cuisine
    cui = pd.DataFrame()
    for i in range(20):
        try:
            res = search(API_KEY,cuisine,loc,cat,50*i,price)['businesses']
    #         if res == []:
    #             break
            k = json_normalize(res)
            cui = pd.concat([cui,k],ignore_index=True)
        except:
            continue
    
#     fact = pd.DataFrame()
#     for x in cui.index:
#         for e in cui.loc[x,'categories']:
#             if e['title'] == cat:
#                 fact = pd.concat([fact,cui[cui.index==x]])
#                 break
    cui.to_csv(cuisine+'_1_'+loc+'_'+str(price)+'.csv',encoding='utf-8')
    print(cuisine,loc,price,len(cui))

# Fast Food

In [19]:
res = search(API_KEY,'fast food','Manhattan','hotdogs',0,1)

In [20]:
res

{u'businesses': [{u'alias': u'chick-fil-a-new-york-6',
   u'categories': [{u'alias': u'hotdogs', u'title': u'Fast Food'}],
   u'coordinates': {u'latitude': 40.751739, u'longitude': -73.986051},
   u'display_phone': u'(212) 704-9920',
   u'distance': 1136.9888965335217,
   u'id': u'k7n_31GE1Yq1KD77_DO1Tw',
   u'image_url': u'https://s3-media2.fl.yelpcdn.com/bphoto/QfD8uORSK89xdE_eq6wEbA/o.jpg',
   u'is_closed': False,
   u'location': {u'address1': u'1000 Avenue Of The Americas',
    u'address2': u'',
    u'address3': u'',
    u'city': u'New York',
    u'country': u'US',
    u'display_address': [u'1000 Avenue Of The Americas',
     u'New York, NY 10018'],
    u'state': u'NY',
    u'zip_code': u'10018'},
   u'name': u'Chick-fil-A',
   u'phone': u'+12127049920',
   u'price': u'$',
   u'rating': 4.0,
   u'review_count': 492,
   u'transactions': [],
   u'url': u'https://www.yelp.com/biz/chick-fil-a-new-york-6?adjust_creative=Uu3HbS4pg129T9EGGOQ4HQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_b

In [24]:
for i in ['Manhattan','Brooklyn','Bronx','Queens','Staten Island']:
    for j in [1,2,3,4]:
        getrestaurant('fast food',loc=i,cat='hotdogs',price=j)

fast food Manhattan 1 798
fast food Manhattan 2 180
fast food Manhattan 3 4
fast food Manhattan 4 4
fast food Brooklyn 1 667
fast food Brooklyn 2 155
fast food Brooklyn 3 4
fast food Brooklyn 4 4
fast food Bronx 1 629
fast food Bronx 2 154
fast food Bronx 3 4
fast food Bronx 4 3
fast food Queens 1 1000
fast food Queens 2 254
fast food Queens 3 3
fast food Queens 4 3
fast food Staten Island 1 357
fast food Staten Island 2 76
fast food Staten Island 3 5
fast food Staten Island 4 5


In [11]:
getrestaurant("Latin American",loc='New York',price=[2])

Latin American New York [2] 449


In [12]:
getrestaurant("Latin American",loc='New York',price=[1,3,4])

Latin American New York [1, 3, 4] 764


In [46]:
search(API_KEY,'Japanese',"Queens",50*2,"Japanese",4)

{u'businesses': [],
 u'region': {u'center': {u'latitude': 40.73146603137887,
   u'longitude': -73.85284423828125}},
 u'total': 100}

In [51]:
for i in ['Manhattan','Brooklyn','Bronx','Queens','Staten Island']:
    for j in [1,2,3,4]:
        getrestaurant('Japanese',loc=i,price=j)

Japanese Manhattan 1 before 241 after 153
Japanese Manhattan 2 before 599 after 448
Japanese Manhattan 3 before 198 after 114
Japanese Manhattan 4 before 91 after 62
Japanese Brooklyn 1 before 221 after 138
Japanese Brooklyn 2 before 949 after 695
Japanese Brooklyn 3 before 190 after 110
Japanese Brooklyn 4 before 93 after 63
Japanese Bronx 1 before 210 after 132
Japanese Bronx 2 before 899 after 675
Japanese Bronx 3 before 199 after 117
Japanese Bronx 4 before 95 after 64
Japanese Queens 1 before 288 after 193
Japanese Queens 2 before 1000 after 812
Japanese Queens 3 before 229 after 137
Japanese Queens 4 before 100 after 68
Japanese Staten Island 1 before 105 after 69
Japanese Staten Island 2 before 610 after 404
Japanese Staten Island 3 before 76 after 48
Japanese Staten Island 4 before 39 after 28


In [26]:
import os

In [27]:
dfs = pd.DataFrame()
for filename in os.listdir('.'):
    if 'fast food' in filename:
        print("Now reading: %s"%(filename))
        df = pd.read_csv(filename, engine='python')
        # append category name used for query
        dfs = pd.concat([dfs,df],ignore_index=True)

Now reading: fast food_Manhattan_3.csv
Now reading: fast food_Queens_3.csv
Now reading: fast food_Staten Island_2.csv
Now reading: fast food_Staten Island_3.csv
Now reading: fast food_Queens_2.csv
Now reading: fast food_Manhattan_2.csv
Now reading: fast food_Staten Island_1.csv
Now reading: fast food_Queens_1.csv
Now reading: fast food_Manhattan_1.csv
Now reading: fast food_Staten Island_4.csv
Now reading: fast food_Manhattan_4.csv
Now reading: fast food_Queens_4.csv
Now reading: fast food_Brooklyn_2.csv
Now reading: fast food_Brooklyn_3.csv
Now reading: fast food_Brooklyn_1.csv
Now reading: fast food_Bronx_4.csv
Now reading: fast food_Brooklyn_4.csv
Now reading: fast food_Bronx_1.csv
Now reading: fast food_Bronx_2.csv
Now reading: fast food_Bronx_3.csv


In [28]:
len(dfs)

4309

In [29]:
t = dfs.drop_duplicates(['id'])

In [30]:
len(t)

1407

In [31]:
t.head()

,Unnamed: 0,alias,categories,coordinates.latitude,coordinates.longitude,display_phone,distance,id,image_url,is_closed,...,location.display_address,location.state,location.zip_code,name,phone,price,rating,review_count,transactions,url
0,0,chick-fil-a-englewood-3,"[{u'alias': u'hotdogs', u'title': u'Fast Food'}]",40.894990,-73.976200,(201) 227-2050,14987.545933,vIfNf5_fBbRhOB2MfG8e8w,https://s3-media1.fl.yelpcdn.com/bphoto/ZJXKpP...,False,...,"[2 Nathaniel Pl, Englewood, NJ 07631]",NJ,7631,Chick-fil-A,1.201227e+10,$$$,3.5,11,[],https://www.yelp.com/biz/chick-fil-a-englewood...
1,1,popeyes-louisiana-kitchen-brooklyn-14,"[{u'alias': u'hotdogs', u'title': u'Fast Food'...",40.699090,-73.917126,(718) 381-9681,8544.197692,M0K_65YKPUzSVAkKmXMdOA,https://s3-media1.fl.yelpcdn.com/bphoto/884y7O...,False,...,"[1465 Myrtle Ave, Brooklyn, NY 11237]",NY,11237,Popeyes Louisiana Kitchen,1.718382e+10,$$$,3.0,20,[],https://www.yelp.com/biz/popeyes-louisiana-kit...
2,2,mcdonalds-branchburg,"[{u'alias': u'hotdogs', u'title': u'Fast Food'}]",40.568219,-74.683244,(908) 595-1381,63159.514463,7XM_kzCFb60bntPNlwuJQg,https://s3-media4.fl.yelpcdn.com/bphoto/1hYriG...,False,...,"[944 Rt 202 S, Branchburg, NJ 08876]",NJ,8876,McDonald's,1.908595e+10,$$$,2.0,16,[],https://www.yelp.com/biz/mcdonalds-branchburg?...
3,3,sbarro-brooklyn-3,"[{u'alias': u'italian', u'title': u'Italian'},...",40.609730,-73.921840,NaN,17436.894383,h6Ndv5mSLG_FgzKiCv3Rmg,https://s3-media3.fl.yelpcdn.com/bphoto/T23ev-...,False,...,"[2509 Flatbush Ave, Brooklyn, NY 11234]",NY,11234,Sbarro,NaN,$$$,3.0,4,[],https://www.yelp.com/biz/sbarro-brooklyn-3?adj...
7,0,chipotle-mexican-grill-staten-island-3,"[{u'alias': u'mexican', u'title': u'Mexican'},...",40.567917,-74.111436,(718) 980-1742,3168.767037,LqbTY7FFAIJPMqdKpuJf7Q,https://s3-media2.fl.yelpcdn.com/bphoto/wAOS12...,False,...,"[2602 Hylan Blvd, Staten Island, NY 10309]",NY,10309,Chipotle Mexican Grill,1.718980e+10,$$,2.5,35,[],https://www.yelp.com/biz/chipotle-mexican-gril...


In [32]:
t = t.drop(['Unnamed: 0'],axis=1)

In [36]:
t.to_csv('FastFood.csv',index=False)

In [39]:
t.head(1)

,alias,categories,coordinates.latitude,coordinates.longitude,display_phone,distance,id,image_url,is_closed,location.address1,...,location.display_address,location.state,location.zip_code,name,phone,price,rating,review_count,transactions,url
0,chick-fil-a-englewood-3,"[{u'alias': u'hotdogs', u'title': u'Fast Food'}]",40.89499,-73.9762,(201) 227-2050,14987.545933,vIfNf5_fBbRhOB2MfG8e8w,https://s3-media1.fl.yelpcdn.com/bphoto/ZJXKpP...,False,2 Nathaniel Pl,...,"[2 Nathaniel Pl, Englewood, NJ 07631]",NJ,7631,Chick-fil-A,1.201227e+10,$$$,3.5,11,[],https://www.yelp.com/biz/chick-fil-a-englewood...


In [37]:
tt = pd.read_csv('FastFood.csv')

In [38]:
tt.head()

,alias,categories,coordinates.latitude,coordinates.longitude,display_phone,distance,id,image_url,is_closed,location.address1,...,location.display_address,location.state,location.zip_code,name,phone,price,rating,review_count,transactions,url
0,chick-fil-a-englewood-3,"[{u'alias': u'hotdogs', u'title': u'Fast Food'}]",40.894990,-73.976200,(201) 227-2050,14987.545933,vIfNf5_fBbRhOB2MfG8e8w,https://s3-media1.fl.yelpcdn.com/bphoto/ZJXKpP...,False,2 Nathaniel Pl,...,"[2 Nathaniel Pl, Englewood, NJ 07631]",NJ,7631,Chick-fil-A,1.201227e+10,$$$,3.5,11,[],https://www.yelp.com/biz/chick-fil-a-englewood...
1,popeyes-louisiana-kitchen-brooklyn-14,"[{u'alias': u'hotdogs', u'title': u'Fast Food'...",40.699090,-73.917126,(718) 381-9681,8544.197692,M0K_65YKPUzSVAkKmXMdOA,https://s3-media1.fl.yelpcdn.com/bphoto/884y7O...,False,1465 Myrtle Ave,...,"[1465 Myrtle Ave, Brooklyn, NY 11237]",NY,11237,Popeyes Louisiana Kitchen,1.718382e+10,$$$,3.0,20,[],https://www.yelp.com/biz/popeyes-louisiana-kit...
2,mcdonalds-branchburg,"[{u'alias': u'hotdogs', u'title': u'Fast Food'}]",40.568219,-74.683244,(908) 595-1381,63159.514463,7XM_kzCFb60bntPNlwuJQg,https://s3-media4.fl.yelpcdn.com/bphoto/1hYriG...,False,944 Rt 202 S,...,"[944 Rt 202 S, Branchburg, NJ 08876]",NJ,8876,McDonald's,1.908595e+10,$$$,2.0,16,[],https://www.yelp.com/biz/mcdonalds-branchburg?...
3,sbarro-brooklyn-3,"[{u'alias': u'italian', u'title': u'Italian'},...",40.609730,-73.921840,NaN,17436.894383,h6Ndv5mSLG_FgzKiCv3Rmg,https://s3-media3.fl.yelpcdn.com/bphoto/T23ev-...,False,2509 Flatbush Ave,...,"[2509 Flatbush Ave, Brooklyn, NY 11234]",NY,11234,Sbarro,NaN,$$$,3.0,4,[],https://www.yelp.com/biz/sbarro-brooklyn-3?adj...
4,chipotle-mexican-grill-staten-island-3,"[{u'alias': u'mexican', u'title': u'Mexican'},...",40.567917,-74.111436,(718) 980-1742,3168.767037,LqbTY7FFAIJPMqdKpuJf7Q,https://s3-media2.fl.yelpcdn.com/bphoto/wAOS12...,False,2602 Hylan Blvd,...,"[2602 Hylan Blvd, Staten Island, NY 10309]",NY,10309,Chipotle Mexican Grill,1.718980e+10,$$,2.5,35,[],https://www.yelp.com/biz/chipotle-mexican-gril...


In [76]:
len(t),len(tt)

(2203, 1248)

In [77]:
a = pd.concat([t,tt],ignore_index=True)

In [79]:
aa = a.drop_duplicates(['id'])

In [80]:
len(aa)

2269

In [81]:
aa.to_csv('Japanese_new.csv')

In [21]:
j = pd.read_csv('Data/asian/Japanese.csv')

In [22]:
j.head()

,Unnamed: 0,Unnamed: 0.1,alias,categories,coordinates.latitude,coordinates.longitude,display_phone,distance,id,image_url,...,location.display_address,location.state,location.zip_code,name,phone,price,rating,review_count,transactions,url
0,0,NaN,aoyu-sushi-bronx,"[{u'alias': u'japanese', u'title': u'Japanese'...",40.886163,-73.909696,(718) 884-6633,4963.236585,5qF6oqYOeivUbsVf1MsQAA,https://s3-media3.fl.yelpcdn.com/bphoto/zeXU2E...,...,"[3532 A Johnson Ave, Bronx, NY 10463]",NY,10463.0,Aoyu Sushi,1.718885e+10,$$,4.0,192,[pickup],https://www.yelp.com/biz/aoyu-sushi-bronx?adju...
1,1,NaN,tokyo-house-bronx,"[{u'alias': u'japanese', u'title': u'Japanese'}]",40.905621,-73.904083,(718) 601-6877,6549.826195,5kufgNH-OuUBDPaDQIEtzw,https://s3-media2.fl.yelpcdn.com/bphoto/OhKEyc...,...,"[5648 Riverdale Ave, Bronx, NY 10471]",NY,10471.0,Tokyo House,1.718602e+10,$$,4.0,60,"[delivery, pickup]",https://www.yelp.com/biz/tokyo-house-bronx?adj...
2,2,NaN,kai-sushi-asian-fusion-bronx,"[{u'alias': u'asianfusion', u'title': u'Asian ...",40.843620,-73.845650,(718) 863-7900,2355.603389,rN347NNwQ0hDSlbedzI9BQ,https://s3-media1.fl.yelpcdn.com/bphoto/oxO3r9...,...,"[1475 Williamsbridge Rd, Bronx, NY 10461]",NY,10461.0,Kai Sushi Asian Fusion,1.718864e+10,$$,4.0,183,"[delivery, pickup, restaurant_reservation]",https://www.yelp.com/biz/kai-sushi-asian-fusio...
3,3,NaN,kiraku-sushi-bronx,"[{u'alias': u'sushi', u'title': u'Sushi Bars'}...",40.853288,-73.853494,(718) 379-8866,1521.989251,kCk2uqmmJBQ80iLuncNhlQ,https://s3-media2.fl.yelpcdn.com/bphoto/cc65KE...,...,"[1948 Williamsbridge Rd, Bronx, NY 10461]",NY,10461.0,Kiraku Sushi,1.718380e+10,$$,3.5,57,[],https://www.yelp.com/biz/kiraku-sushi-bronx?ad...
4,4,NaN,palace-of-japan-bronx,"[{u'alias': u'japanese', u'title': u'Japanese'...",40.886120,-73.910000,(718) 543-8833,4970.343846,py1zfy48JMV6tMZRvgxMKA,https://s3-media1.fl.yelpcdn.com/bphoto/5iLrHS...,...,"[3505 Johnson Ave, Bronx, NY 10463]",NY,10463.0,Palace of Japan,1.718544e+10,$$,3.5,159,"[delivery, pickup, restaurant_reservation]",https://www.yelp.com/biz/palace-of-japan-bronx...


In [23]:
len(j)

2269

In [26]:
cat = set(['Japanese','Sushi Bars','Japanese Curry','Izakaya'])

In [41]:
fact = pd.DataFrame()
for x in t.index:
    print(t.loc[x,'categories'])
    for e in t.loc[x,'categories']:
        print(e)
        if e['title'] in cat:
            fact = pd.concat([fact,t[t.index==x]])
            break
# fact.to_csv(cuisine+'_'+loc+'_'+str(price)+'.csv',encoding='utf-8')

[{u'alias': u'japanese', u'title': u'Japanese'}, {u'alias': u'sushi', u'title': u'Sushi Bars'}, {u'alias': u'salad', u'title': u'Salad'}]
[


TypeError: string indices must be integers, not str